In [ ]:
class MyDataset(...):
  def init(self, path2data):
    paths2images = [os.path.join(path2data, image) for image in os.listdir(path2data)]
    texts = [text for text in ...]

  def getitem(self, i):
    image = np.array(Image.open(paths2images[i]))
    # preprocess your photo
    image_as_vector = get_vector_from_image(image)

    # Same with text
    text_as_vector = ...

    return image_as_vector, text_as_vector

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install torchmetrics
!pip install segmentation_models_pytorch



import sys
import os, cv2
import numpy as np
import pandas as pd
import random, tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as album
from torchmetrics import JaccardIndex
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils
import gc


import PIL
from PIL import Image

import os, cv2
import numpy as np
import pandas as pd
import random, tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as album

import PIL
from PIL import Image
import segmentation_models_pytorch as smp


# Perform one hot encoding on label
def one_hot_encode(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes
    # Arguments
        label: The 2D array segmentation image label
        label_values

    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)

    return semantic_map

# Perform reverse one-hot-encoding on labels / preds
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.
    # Arguments
        image: The one-hot format image

    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified
        class key.
    """
    x = np.argmax(image, axis = -1)
    return x

# Perform colour coding on the reverse-one-hot outputs
def colour_code_segmentation(image, label_values):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.
    # Arguments
        image: single channel array where each value represents the class key.
        label_values

    # Returns
        Colour coded image for segmentation visualization
    """
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x
class DefectsDataset(torch.utils.data.Dataset):

    """Read images, apply augmentation and preprocessing transformations.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_rgb_values (list): RGB values of select classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing
            (e.g. normalization, shape manipulation, etc.)

    """

    def __init__(
            self,
            images_dir: str,
            masks_dir: str,
            class_rgb_values=None,
            augmentation=None,
            preprocessing=None,
            mask_format = 'np',
            size=-1
    ):

        self.image_paths = [os.path.join(images_dir, image_id) for image_id in sorted(os.listdir(images_dir))]#[:size]
        self.mask_paths = [os.path.join(masks_dir, image_id) for image_id in sorted(os.listdir(masks_dir))]#[:size]

        self.class_rgb_values = class_rgb_values
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        self.mask_format = mask_format

    def __getitem__(self, i):

        # read images and masks
        #image = cv2.cvtColor(cv2.imread(self.image_paths[i]), cv2.COLOR_BGR2RGB)
        #mask = cv2.cvtColor(cv2.imread(self.mask_paths[i]), cv2.COLOR_BGR2RGB)

        image = np.array(Image.open(self.image_paths[i]))
        if self.mask_format == 'np':
          mask = np.load(self.mask_paths[i])
        else:
          mask = np.array(Image.open(self.mask_paths[i]))


        # one-hot-encode the mask
        mask = one_hot_encode(mask, self.class_rgb_values).astype('float')

        # apply augmentations
        if self.augmentation:
            #print(self.image_paths[i])
            #print(i)
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask

    def __len__(self):
        # return length of
        return len(self.image_paths)

def visualize(**images):
    """
    Plot images in one row
    """
    n_images = len(images)
    plt.figure(figsize=(20,8))
    for idx, (name, image) in enumerate(images.items()):
        plt.subplot(1, n_images, idx + 1)
        plt.xticks([]);
        plt.yticks([])
        # get title from the parameter names
        plt.title(name.replace('_',' ').title(), fontsize=20)
        plt.imshow(image)
    plt.show()

#def get_training_augmentation():
#    test_transform = [
#        album.PadIfNeeded(min_height=2368, min_width=2368, always_apply=True, border_mode=0),
#    ]
#    return album.Compose(test_transform)

# Here you can specify constant or probabilistic augmentation
def get_training_augmentation():
    train_transform = [
        album.RandomCrop(height=416, width=416, always_apply=True),
        album.OneOf(
            [
                album.HorizontalFlip(p=1),
                album.VerticalFlip(p=1),
                album.RandomRotate90(p=1),
            ],
            p=0.75,
        ),
    ]
    return album.Compose(train_transform)

# Here you can specify constant or probabilistic augmentation
def get_validation_augmentation():
    # Add sufficient padding to ensure image is divisible by 32
    test_transform = [
        album.PadIfNeeded(min_height=3040, min_width=4032, always_apply=True, border_mode=1),
    ]
    return album.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn=None):
    """Construct preprocessing transform
    Args:
        preprocessing_fn (callable): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    """
    _transform = []
    if preprocessing_fn:
        _transform.append(album.Lambda(image=preprocessing_fn))
    _transform.append(album.Lambda(image=to_tensor, mask=to_tensor))

    return album.Compose(_transform)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class_names = ['background', 'defect']
class_rgb_values = [[0, 255, 0], [255, 0, 0]]

In [ ]:
ENCODER = 'resnet50'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = class_names
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation

# create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=len(CLASSES),
    activation=ACTIVATION,
)
#model = torch.load(f'/content/drive/MyDrive/Defects/Models/best_model_10_{0}_epoch.pth')

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug"

# Directories of train, test etc.
x_train_dir = os.path.join(DATA_DIR, 'TRAIN/original')
y_train_dir = os.path.join(DATA_DIR, 'TRAIN/segment')

x_valid_dir = os.path.join(DATA_DIR, 'VAL/original')
y_valid_dir = os.path.join(DATA_DIR, 'VAL/segment')

x_test_dir = os.path.join(DATA_DIR, 'TEST/original')
y_test_dir = os.path.join(DATA_DIR, 'TEST/segment')

# Here you can specify constant or probabilistic augmentation
# def get_training_augmentation_():
#     test_transform = [
#        #album.PadIfNeeded(min_height=288, min_width=288, always_apply=True, border_mode=0),
#        album.PadIfNeeded(min_height=416, min_width=416, always_apply=True, border_mode=0),
#     ]
#     return album.Compose(test_transform)

def get_validation_augmentation_():
    # Add sufficient padding to ensure image is divisible by 32
    test_transform = [
        #album.PadIfNeeded(min_height=288, min_width=288, always_apply=True, border_mode=0),
        album.PadIfNeeded(min_height=416, min_width=416, always_apply=True, border_mode=0),
    ]
    return album.Compose(test_transform)

train_dataset = DefectsDataset(
    x_train_dir, y_train_dir,
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn),
    class_rgb_values=class_rgb_values,
    mask_format='png'
)

valid_dataset = DefectsDataset(
    x_valid_dir, y_valid_dir,
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocessing_fn),
    class_rgb_values=class_rgb_values,
    mask_format='png'
)

# Get train and val data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=1)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
for i in range(len(train_dataset)):
  x, y = train_dataset[0]
  print(x.shape)
  print(y.shape)
  print('------------')

(3, 416, 416)
(2, 416, 416)
------------
(3, 416, 416)
(2, 416, 416)
------------
(3, 416, 416)
(2, 416, 416)
------------
(3, 416, 416)
(2, 416, 416)
------------
(3, 416, 416)
(2, 416, 416)
------------
(3, 416, 416)
(2, 416, 416)
------------
(3, 416, 416)
(2, 416, 416)
------------
(3, 416, 416)
(2, 416, 416)
------------
(3, 416, 416)
(2, 416, 416)
------------


In [ ]:
torch.Tensor(train_dataset[0][0]).shape#[None, :, :, :].shape

torch.Size([3, 416, 416])

In [ ]:
model(torch.Tensor(train_dataset[-2][0])[None, :, :, :].cuda()).shape

torch.Size([1, 2, 416, 416])

In [ ]:
x.shape

(3, 288, 288)

In [ ]:
y.shape

(2, 288, 288)

In [ ]:
train_dataset[-1][1].shape

(2, 416, 416)

In [ ]:
len(valid_loader)

2

In [ ]:
from tqdm.auto import tqdm as tqdm
for batch in tqdm(train_loader):
  X, y = batch
  print(X.shape)
  print(y.shape)
  output = model(X.to('cuda'))
  print(output.shape)
  print('----------')

  0%|          | 0/4 [00:00<?, ?it/s]

/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/1.png
0
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/16.png
2
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/IMG_0370.png
4
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/IMG_0370.png
4
torch.Size([2, 3, 416, 416])
torch.Size([2, 2, 416, 416])
torch.Size([2, 2, 416, 416])
----------
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/IMG_0373.png
5
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/IMG_0377.png
7
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/2.png
3
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/2.png
3
torch.Size([2, 3, 416, 416])
torch.Size([2, 2, 416, 416])
torch.Size([2, 2, 416, 416])
----------
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/1.png
0
/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/IMG_0376.png
6
torch.Size([2, 3, 416, 416])

In [ ]:
        image = np.array(Image.open('/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/original/IMG_0360.png'))
        mask = np.array(Image.open('/content/drive/MyDrive/Кирпичи дефекты/Data/no_aug/TRAIN/segment/IMG_0360.png'))

In [ ]:
image.shape

(3024, 4032, 3)

In [ ]:
mask.shape

(2268, 4032, 3)

In [ ]:
from segmentation_models_pytorch import utils
TRAINING = True

# Set num of epochs
EPOCHS = 300

# Set device: `cuda` or `cpu`
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define loss function
#loss = smp.utils.losses.DiceLoss()#DiceLoss_()#
loss = torch.nn.BCELoss()

# define metrics
metrics = [
    #IoU_(),
    smp.utils.metrics.IoU(threshold=0.5),
    smp.utils.metrics.Fscore(threshold=0.5),
    smp.utils.metrics.Accuracy(threshold=0.5),
    smp.utils.metrics.Recall(threshold=0.5),
    smp.utils.metrics.Precision(threshold=0.5),
    #Precision(),
    #Recall(),
    #F1(),
    #Accuracy()
    #smp.utils.metrics.IoU(threshold=0.5),
]

# define optimizer
optimizer = torch.optim.Adam([
    dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model,
    loss=loss,
    metrics=metrics,
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model,
    loss=loss,
    metrics=metrics,
    device=DEVICE,
    verbose=True,
)

In [ ]:
!pip install wandb
import wandb
wandb.login(key='028f2bb73f8d6aa6ae54ccfb3d22b397309bbdd0')

wandb: Currently logged in as: s_artamonov. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%%time
wandb.init(
      # Set the project where this run will be logged
      project="Brics_unet",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{1}_300_epoch_bce",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0001,
      "architecture": "Unet-_400",
      "dataset": "Own",
      "epochs": 300,
      "comment": 'proba_augm400'
      })

CPU times: user 131 ms, sys: 13.5 ms, total: 144 ms
Wall time: 6.7 s


In [ ]:
loss.__name__ = 'bce'

In [ ]:
if TRAINING:

    best_iou_score = 0.0
    train_logs_list, valid_logs_list = [], []

    for i in range(0, 300):

        # Perform training & validation
        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)
        train_logs_list.append(train_logs)
        valid_logs_list.append(valid_logs)

        #val_loss = valid_logs['dice_loss']
        #train_loss = train_logs['dice_loss']
        val_loss = valid_logs['bce']
        train_loss = train_logs['bce']
        val_iou = valid_logs['iou_score']
        train_iou = train_logs['iou_score']


        wandb.log({"train/Loss": train_loss,
                   "val/Loss": val_loss,
                   "train/IoU": train_iou,
                   "val/IoU": val_iou,
                   "train/precision": train_logs['precision'],
                   "val/precision": valid_logs['precision'],
                   "train/recall": train_logs['recall'],
                   "val/recall": valid_logs['recall'],
                   "train/f1": train_logs['fscore'],
                   "val/f1": valid_logs['fscore'],
                   "train/acc": train_logs['accuracy'],
                   "val/acc": valid_logs['accuracy']})
        if i%5 == 0:
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Кирпичи дефекты/Models/Unet/Experiment_1/model_{i+1}_epoch.pth')
        # # Save model if a better val IoU score is obtained
        if best_iou_score < valid_logs['iou_score']:
             best_iou_score = valid_logs['iou_score']
             torch.save(model, '/content/drive/MyDrive/Кирпичи дефекты/Models/Unet/Experiment_1/best_model.pth')
             print('Model saved!')


Epoch: 0
valid: 100%|██████████| 2/2 [00:06<00:00,  3.41s/it, bce - 0.6982, iou_score - 0.4643, fscore - 0.6339, accuracy - 0.4812, recall - 0.8998, precision - 0.4894]
Model saved!

Epoch: 1
valid: 100%|██████████| 2/2 [00:08<00:00,  4.16s/it, bce - 0.781, iou_score - 0.3862, fscore - 0.5551, accuracy - 0.3945, recall - 0.762, precision - 0.4371]

Epoch: 2
valid: 100%|██████████| 2/2 [00:06<00:00,  3.37s/it, bce - 0.8254, iou_score - 0.3796, fscore - 0.5481, accuracy - 0.395, recall - 0.7408, precision - 0.4356]

Epoch: 3
valid: 100%|██████████| 2/2 [00:06<00:00,  3.21s/it, bce - 0.7802, iou_score - 0.3835, fscore - 0.5525, accuracy - 0.4244, recall - 0.7175, precision - 0.4501]

Epoch: 4
valid: 100%|██████████| 2/2 [00:06<00:00,  3.36s/it, bce - 0.7745, iou_score - 0.3993, fscore - 0.5683, accuracy - 0.4575, recall - 0.7218, precision - 0.4696]

Epoch: 5
valid: 100%|██████████| 2/2 [00:08<00:00,  4.01s/it, bce - 0.7609, iou_score - 0.4282, fscore - 0.596, accuracy - 0.5119, recall -